In [ ]:
#!jupyter nbconvert --to=python plot_Chlorophyll_FESOM_python3.ipynb

In [ ]:
class plot_Chl:
    '''
    class plot_Chl(resultpath,varname,years,mesh,runid,depth,mapproj,label,units,convert=False)
    
    Plot chlorophyll data to a specified color scale using BoundaryNorm()
    
    convert: set True to convert FESOM output [mmol ChlA/m3] to [mg ChlA/m3]
    '''
    
    def __init__(self,resultpath,varname,years,mesh,runid,depth,mapproj,label,units,convert=False):
        
        self.resultpath = resultpath
        self.varname = varname
        self.years = years
        self.mesh = mesh
        self.runid = runid
        self.depth = depth
        self.mapproj = mapproj
        self.label = label
        self.units = units
        self.convert = convert
        
        import matplotlib.pyplot as plt
        import matplotlib.colors as colors
        import numpy as np
        from scipy.interpolate import griddata
        import cartopy.crs as ccrs

        import pyfesom2 as pf
        from F2R_sensitivity_toolbox import get_seamask_WOA, mygrid
                
        if(self.mapproj == 'rob'): self.projection = ccrs.Robinson()
        elif(self.mapproj == 'pc'): self.projection = ccrs.PlateCarree()
        elif(self.mapproj == 'nps'): self.projection = ccrs.NorthPolarStereo()
        else: raise ValueError('Map projection not supported!')
            
        
        Chl = pf.get_data(self.resultpath, self.varname, 
                     self.years,
                     self.mesh, runid=self.runid, how="mean", compute=True, depth=self.depth)

        # convert FESOM CHl.a data -------------------------------------------------------------------------------------
        #Chl.a : 893.509 g/mol
        if self.convert:
            conv = 893.509
            Chl = Chl * conv
            self.units = 'Chl.a [mg m$^{-3}$]'
            print('\n***\nFESOM data was CONVERTED, assuming [mmol ChlA/m3] to [mg ChlA/m3]\n***')
        else:
            print('\n***\nFESOM data was NOT CONVERTED, assuming [mg ChlA/m3]\n***')
    
        # interpolation
        lat         = np.arange(-89.5,90.,1.)
        lon         = np.arange(-179.5,180.,1.)
        latdic, londic = np.meshgrid(lat, lon)

        Chl_interp_raw = pf.fesom2regular(
                        data = Chl,
                        mesh = mesh,
                        lons = londic, 
                        lats = latdic)
        
        # plotting
        levels = np.array([0,0.01,0.02,0.03,0.04,0.05,0.06,0.07,0.08,0.09,
                                   0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,
                                   1,2,3,4,5,7])
        ticks = [0,0.01,0.03,0.05,0.07,0.1,0.3,0.5,0.7,1,3,5,7]
        ticks_label = ['0','0.01','0.03','0.05','0.07','0.1','0.3','0.5','0.7','1','3','5','7']    



        fig, axes = plt.subplots(1,1, 
                 subplot_kw=dict(projection=self.projection),
                 gridspec_kw={'hspace': 0.001, 'wspace': 0.1},
                 figsize=(15,12))

        m2 = axes
        f2 = m2.pcolormesh(londic, latdic, Chl_interp_raw, 
                           transform = ccrs.PlateCarree(),
                           norm=colors.BoundaryNorm(boundaries=levels, ncolors=256))
        
        m2.set_title(label, fontsize=16)

        # add land features, coastline and grid
        #m2.coastlines(resolution='110m', color='black', linewidth=1)
        mygrid(m2, grid=True)
        
        if(self.mapproj == 'nps'):
            plot_extent = [-180, 180, 70, 90]
            m2.set_extent(plot_extent, crs=ccrs.PlateCarree())

        # add colorbar below figure
        cbar = fig.colorbar(f2,
                            ax = axes, 
                            #location ='bottom',
                            orientation = 'horizontal',
                            ticks = ticks,
                            fraction=0.1, pad=0.1) 
        #cbar.ax.tick_params(labelsize=14)
        cbar.ax.set_xticklabels(ticks_label, fontsize=16) 
        cbar.set_label(units, fontsize=16)
        
        

In [ ]:
### TESTING ###
if __name__ == "__main__":
    
    import numpy as np
    import pyfesom2 as pf
    
    runid      =  'fesom'
    resultpath = '/work/ollie/mozeisin/results/f2r1.2/' + 'mo20'
    savepath = '/home/ollie/mozeisin/evaluation/mo_files/'

    meshpath = '/work/ollie/mozeisin/mesh/mesh_fesom2.0/core2_meanz'
    mesh = pf.load_mesh(meshpath)
    
    depth= 0
    
    first_year = 1965
    last_year  = 1970
    years_last10 = np.arange(last_year-10,last_year+1,1)
    
    label='Diatom Chlorophyll {0}-{1} at {2} m'.format(last_year-10,last_year,depth)
    units = 'Chl.a [mg m$^{-3}$]' 
    mapproj = 'nps'

    plot_Chl(resultpath,'DiaChl',
             years_last10,
             mesh,runid,depth,mapproj,label,units)